Deconstructing the Class Statement

- The class statement starts a block of code and creates a new namespace. All namespace changes in the block, e.g. assignment and function definitions, are made in that new namespace. 
Finally it adds the class name to the namespace where the class statement appears.

- Instances of a class are created by calling the class: ClassName() or ClassName(args).

- ClassName.__init__(<new object>, ...) is called automatically, and is passed the instance of the class already created by a call to the __new__ method.

- Accessing an attribute method_name on a class instance returns a method object, if method_name references a method (in ClassName or its superclasses). A method object binds the class instance as the first argument to the method.


In [1]:

class Number:  # In Python 2.x use "class Number(object):"
    """A number class."""
    __version__ = '1.0'
    
    def __init__(self, amount):
        self.amount = amount
    
    def add(self, value):
        """Add a value to the number."""
        print('Call: add({!r}, {})'.format(self, value))
        return self.amount + value

In [2]:
Number

__main__.Number

In [3]:
Number.__version__

'1.0'

In [4]:
Number.__doc__

'A number class.'

In [5]:
help(Number)

Help on class Number in module __main__:

class Number(builtins.object)
 |  A number class.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, amount)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  add(self, value)
 |      Add a value to the number.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [6]:
Number.__init__

<function __main__.Number.__init__>

In [7]:
Number.add

<function __main__.Number.add>

In [8]:
dir(Number)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__version__',
 '__weakref__',
 'add']

In [9]:

def dir_public(obj):
    return [n for n in dir(obj) if not n.startswith('__')]

In [10]:
dir_public(Number)

['add']

In [11]:
number2=Number(2)

In [12]:
number2.amount

2

In [13]:
number2

In [14]:
number2.__init__
number2.add

<bound method Number.add of <__main__.Number object at 0x1041a4978>>

In [15]:
dir_public(number2)

['add', 'amount']

In [16]:
set(dir(number2)) ^ set(dir(Number)) # symmetric_difference


{'amount'}

In [17]:
number2.__dict__

{'amount': 2}

In [18]:
Number.__dict__


mappingproxy({'__doc__': 'A number class.', '__module__': '__main__', 'add': <function Number.add at 0x1041b6510>, '__init__': <function Number.__init__ at 0x1041b6488>, '__dict__': <attribute '__dict__' of 'Number' objects>, '__weakref__': <attribute '__weakref__' of 'Number' objects>, '__version__': '1.0'})

In [19]:
'add' in Number.__dict__

True

In [20]:
number2.add

<bound method Number.add of <__main__.Number object at 0x1041a4978>>

In [21]:
number2.add(3)

Call: add(<__main__.Number object at 0x1041a4978>, 3)


5

Here's some unusual code ahead which will help us think carefully about how Python works.


In [22]:
Number.add

<function __main__.Number.add>

Will this work? Here's the gist of the method add defined above:


In [23]:
def add(self, value):
        return self.amount + value

In [25]:
Number.add(2)


TypeError: add() missing 1 required positional argument: 'value'

In [26]:
Number.add(2, 3)


Call: add(2, 3)


AttributeError: 'int' object has no attribute 'amount'

In [27]:
Number.add(number2, 3)


Call: add(<__main__.Number object at 0x1041a4978>, 3)


5

In [28]:
number2.add(3)


Call: add(<__main__.Number object at 0x1041a4978>, 3)


5

Remember, here's how __init__ was defined above:


In [29]:
def __init__(self, amount):
        self.amount = amount

In [30]:
Number.__init__


<function __main__.Number.__init__>

In [31]:
help(Number.__init__)


Help on function __init__ in module __main__:

__init__(self, amount)
    Initialize self.  See help(type(self)) for accurate signature.



Here's some code that's downright risky, but instructive. You should never need to do this in your code.


In [32]:
def set_double_amount(number, amount):
    number.amount = 2 * amount


In [33]:
Number.__init__ = set_double_amount


In [34]:
Number.__init__


<function __main__.set_double_amount>

In [35]:
help(Number.__init__)


Help on function set_double_amount in module __main__:

set_double_amount(number, amount)



In [36]:
number4 = Number(2)


In [37]:
number4.amount


4

In [38]:
number4.add(5)


Call: add(<__main__.Number object at 0x1042454a8>, 5)


9

In [39]:
number4.__init__


<bound method set_double_amount of <__main__.Number object at 0x1042454a8>>

In [40]:
number2.__init__


<bound method set_double_amount of <__main__.Number object at 0x1041a4978>>

In [41]:
def multiply_by(number, value):
    return number.amount * value

Let's add a mul method. However, I will intentionally make a mistake.


In [42]:
number4.mul = multiply_by


In [43]:
number4.mul


<function __main__.multiply_by>

In [44]:
number4.mul(5)


TypeError: multiply_by() missing 1 required positional argument: 'value'

In [45]:
number4.mul(number4, 5)


20

Where's the mistake?


In [46]:
number10 = Number(5)


In [47]:
number10.mul


AttributeError: 'Number' object has no attribute 'mul'

In [48]:
dir_public(number10)


['add', 'amount']

In [49]:
dir_public(Number)


['add']

In [50]:
dir_public(number4)


['add', 'amount', 'mul']

In [51]:
Number.mul = multiply_by


In [52]:
number10.mul(5)


50

In [53]:
number4.mul(5)


TypeError: multiply_by() missing 1 required positional argument: 'value'

In [54]:
dir_public(number4)


['add', 'amount', 'mul']

In [55]:
number4.__dict__


{'amount': 4, 'mul': <function __main__.multiply_by>}

In [56]:
del number4.mul


In [57]:
number4.__dict__


{'amount': 4}

In [58]:
dir_public(number4)


['add', 'amount', 'mul']

In [59]:
number4.mul


<bound method multiply_by of <__main__.Number object at 0x1042454a8>>

In [60]:
Number.mul


<function __main__.multiply_by>

In [61]:
number4.mul(5)


20

Let's look behind the curtain to see how class instances work in Python.


In [62]:
Number

__main__.Number

In [63]:
number4


In [64]:
Number.add


<function __main__.Number.add>

In [65]:
number4.add


<bound method Number.add of <__main__.Number object at 0x1042454a8>>

Bound methods are handy.


In [66]:
add_to_4 = number4.add


In [67]:
add_to_4(6)


Call: add(<__main__.Number object at 0x1042454a8>, 6)


10

In [68]:
dir_public(number4)


['add', 'amount', 'mul']

In [69]:
dir(number4.add)


['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [70]:
dir_public(number4.add)


[]

In [71]:
set(dir(number4.add)) - set(dir(Number.add))


{'__func__', '__self__'}

In [72]:
number4.add.__self__


In [73]:
number4.add.__self__ is number4


True

In [74]:
number4.add.__func__


<function __main__.Number.add>

In [75]:
number4.add.__func__ is Number.add


True

In [76]:
number4.add.__func__ is number10.add.__func__


True

In [77]:
number4.add(5)


Call: add(<__main__.Number object at 0x1042454a8>, 5)


9

So here's approximately how Python executes number4.add(5):


In [78]:
number4.add.__func__(number4.add.__self__, 5)


Call: add(<__main__.Number object at 0x1042454a8>, 5)


9

 Creating Classes with the type Function

"The class statement is just a way to call a function, take the result, and put it into a namespace." -- Glyph Lefkowitz in *Turtles All The Way Down: Demystifying Deferreds, Decorators, and Declarations* at PyCon 2010
type(name, bases, dict) is the default function that gets called when Python read a class statement.

In [80]:
print(type.__doc__)


type(object_or_name, bases, dict)
type(object) -> the object's type
type(name, bases, dict) -> a new type


Let's use the type function to build a class.


In [81]:
def init(self, amount):
    self.amount = amount

In [82]:
def add(self, value):
    """Add a value to the number."""
    print('Call: add({!r}, {})'.format(self, value))
    return self.amount + value

In [83]:
Number = type(
    'Number', (object,),
    dict(__version__='1.0', __init__=init, add=add))

In [84]:
number3 = Number(3)


In [85]:
type(number3)


__main__.Number

In [86]:
number3.__class__


__main__.Number

In [87]:
number3.__dict__


{'amount': 3}

In [88]:
number3.amount


3

In [89]:
number3.add(4)


Call: add(<__main__.Number object at 0x1041e0438>, 4)


7

Remember, here's the normal way to create a class:


In [90]:
class Number:
    __version__='1.0'
    
    def __init__(self, amount):
        self.amount = amount
    
    def add(self, value):
        return self.amount + value

We can customize how classes get created.
https://docs.python.org/3/reference/datamodel.html#customizing-class-creation

By default, classes are constructed using type(). The class body is executed in a new namespace and the class name is bound locally to the result of type(name, bases, namespace).
The class creation process can be customised by passing the metaclass keyword argument in the class definition line, or by inheriting from an existing class that included such an argument.

The following makes explicit that the metaclass, i.e. the callable that Python should use to create a class, is the built-in function type.

In [91]:
class Number(metaclass=type):
    def __init__(self, amount):
        self.amount = amount